In [31]:
import pandas as pd
import json
import numpy
import os

In [32]:
os.listdir()

['.DS_Store',
 'preprocess_data.ipynb',
 'huggingface_dataset',
 'ss.zip',
 'plug_and_play_nn.ipynb',
 'preprocessed_dataset']

In [33]:
with open("huggingface_dataset/ddxplus/release_evidences.json", "r") as f:
    release_evidences = json.load(f)

In [34]:
with open("huggingface_dataset/ddxplus/release_conditions.json") as f:
    release_conditions = json.load(f)

In [35]:
set_conditions = [a for a in release_conditions.keys()]


In [36]:
set_conditions

['Spontaneous pneumothorax',
 'Cluster headache',
 'Boerhaave',
 'Spontaneous rib fracture',
 'GERD',
 'HIV (initial infection)',
 'Anemia',
 'Viral pharyngitis',
 'Inguinal hernia',
 'Myasthenia gravis',
 'Whooping cough',
 'Anaphylaxis',
 'Epiglottitis',
 'Guillain-Barré syndrome',
 'Acute laryngitis',
 'Croup',
 'PSVT',
 'Atrial fibrillation',
 'Bronchiectasis',
 'Allergic sinusitis',
 'Chagas',
 'Scombroid food poisoning',
 'Myocarditis',
 'Larygospasm',
 'Acute dystonic reactions',
 'Localized edema',
 'SLE',
 'Tuberculosis',
 'Unstable angina',
 'Stable angina',
 'Ebola',
 'Acute otitis media',
 'Panic attack',
 'Bronchospasm / acute asthma exacerbation',
 'Bronchitis',
 'Acute COPD exacerbation / infection',
 'Pulmonary embolism',
 'URTI',
 'Influenza',
 'Pneumonia',
 'Acute rhinosinusitis',
 'Chronic rhinosinusitis',
 'Bronchiolitis',
 'Pulmonary neoplasm',
 'Possible NSTEMI / STEMI',
 'Sarcoidosis',
 'Pancreatic neoplasm',
 'Acute pulmonary edema',
 'Pericarditis']

In [37]:
set_evidences = []

In [38]:
for key, value in release_evidences.items():
    if len(value['possible-values']) > 0:
        for val in value['possible-values']:
            set_evidences.append(f'{key}_@_{val}')
    else:
        set_evidences.append(f'{key}')

In [39]:
print(len(set_evidences))
print(set_evidences)

972
['E_91', 'E_55_@_V_123', 'E_55_@_V_14', 'E_55_@_V_15', 'E_55_@_V_16', 'E_55_@_V_17', 'E_55_@_V_18', 'E_55_@_V_19', 'E_55_@_V_20', 'E_55_@_V_21', 'E_55_@_V_22', 'E_55_@_V_23', 'E_55_@_V_24', 'E_55_@_V_25', 'E_55_@_V_26', 'E_55_@_V_27', 'E_55_@_V_28', 'E_55_@_V_29', 'E_55_@_V_30', 'E_55_@_V_31', 'E_55_@_V_32', 'E_55_@_V_33', 'E_55_@_V_34', 'E_55_@_V_35', 'E_55_@_V_36', 'E_55_@_V_37', 'E_55_@_V_38', 'E_55_@_V_39', 'E_55_@_V_40', 'E_55_@_V_41', 'E_55_@_V_42', 'E_55_@_V_43', 'E_55_@_V_44', 'E_55_@_V_45', 'E_55_@_V_46', 'E_55_@_V_47', 'E_55_@_V_48', 'E_55_@_V_49', 'E_55_@_V_50', 'E_55_@_V_51', 'E_55_@_V_52', 'E_55_@_V_53', 'E_55_@_V_54', 'E_55_@_V_55', 'E_55_@_V_56', 'E_55_@_V_57', 'E_55_@_V_58', 'E_55_@_V_59', 'E_55_@_V_60', 'E_55_@_V_61', 'E_55_@_V_62', 'E_55_@_V_63', 'E_55_@_V_64', 'E_55_@_V_65', 'E_55_@_V_66', 'E_55_@_V_67', 'E_55_@_V_68', 'E_55_@_V_69', 'E_55_@_V_70', 'E_55_@_V_72', 'E_55_@_V_73', 'E_55_@_V_74', 'E_55_@_V_75', 'E_55_@_V_76', 'E_55_@_V_77', 'E_55_@_V_78', 'E_55_@_V_7

In [40]:
dict_data = {}
dict_data['AGE'] = []
dict_data['SEX'] = []

In [41]:
def introduce_conditions(conditions, dict_for_insert):
    for condition in conditions:
        dict_for_insert[condition] = []

In [42]:
def introduce_evidences(evidences, dict_for_insert):
    for evidence in evidences:
        dict_for_insert[evidence] = []

In [43]:
introduce_evidences(set_evidences, dict_data)

In [44]:
introduce_conditions(set_conditions, dict_data)

In [45]:
df = pd.read_csv("huggingface_dataset/ddxplus/train.csv")

In [46]:
len(df)

1025602

In [47]:
median_age = df['AGE'].median()
print(median_age)
most_frequent_sex = df['SEX'].mode()[0]
print(most_frequent_sex)

39.0
F


In [48]:
df['AGE'] = df['AGE'].fillna(median_age)
df['SEX'] = df['SEX'].fillna(most_frequent_sex)

In [49]:
import re
import ast

In [31]:
def convert_to_dict(dataframe, dictionary):
    for index, row in dataframe.iterrows():
        dict_record = {}
        dict_record['AGE'] = row['AGE']
        dict_record['SEX'] = row['SEX']
        evidences_string = row['EVIDENCES']
        evidences_parts = ast.literal_eval(evidences_string)
        differential_diagnosis_string = row['DIFFERENTIAL_DIAGNOSIS']
        differential_diagnosis_parts = ast.literal_eval(differential_diagnosis_string)
        for evidence in evidences_parts:
            dict_record[evidence] = 1
        for evidence in set_evidences:
            if evidence not in dict_record:
                dict_record[evidence] = 0
        for diagnosis_prob in differential_diagnosis_parts:
            dict_record[diagnosis_prob[0]] = diagnosis_prob[1]
        for diagnosis in set_conditions:
            if diagnosis not in dict_record:
                dict_record[diagnosis] = 0
        for key, val in dict_record.items():
            dictionary[key].append(val)

In [50]:
def convert_to_dict(dataframe, dictionary):
    for index, row in dataframe.iterrows():
        dict_record = {}
        dict_record['AGE'] = row['AGE']
        dict_record['SEX'] = row['SEX']
        evidences_string = row['EVIDENCES']
        evidences_parts = ast.literal_eval(evidences_string)
        differential_diagnosis_string = row['PATHOLOGY']
        # differential_diagnosis_parts = ast.literal_eval(differential_diagnosis_string)
        for evidence in evidences_parts:
            dict_record[evidence] = 1
        for evidence in set_evidences:
            if evidence not in dict_record:
                dict_record[evidence] = 0
        dict_record[differential_diagnosis_string] = 1
        for diagnosis in set_conditions:
            if diagnosis not in dict_record:
                dict_record[diagnosis] = 0
        for key, val in dict_record.items():
            dictionary[key].append(val)

In [51]:
convert_to_dict(df, dict_data)

In [52]:
df_preproccessed = pd.DataFrame(dict_data)

In [ ]:
df_preproccessed.to_csv("preprocessed_dataset/preprocessed_train.csv")

In [54]:
dict_data_test = {}
dict_data_test['AGE'] = []
dict_data_test['SEX'] = []

In [55]:
introduce_evidences(set_evidences, dict_data_test)

In [56]:
introduce_conditions(set_conditions, dict_data_test)

In [64]:
df_test = pd.read_csv("huggingface_dataset/ddxplus/test.csv")

In [65]:
median_age = df_test['AGE'].median()
print(median_age)
most_frequent_sex = df_test['SEX'].mode()[0]
print(most_frequent_sex)

39.0
F


In [66]:
df_test['AGE'] = df_test['AGE'].fillna(median_age)
df_test['SEX'] = df_test['SEX'].fillna(most_frequent_sex)

In [67]:
convert_to_dict(df_test, dict_data_test)

In [68]:
df_test_preproccessed = pd.DataFrame(dict_data_test)

In [69]:
df_test_preproccessed.to_csv("preprocessed_dataset/preprocessed_test_nondiff.csv")

In [38]:
dict_data_validate = {}
dict_data_validate['AGE'] = []
dict_data_validate['SEX'] = []

In [39]:
introduce_evidences(set_evidences, dict_data_validate)

In [40]:
introduce_conditions(set_conditions, dict_data_validate)

In [41]:
df_validate = pd.read_csv("huggingface_dataset/ddxplus/validate.csv")

In [42]:
convert_to_dict(df_validate, dict_data_validate)

In [43]:
df_validate_preproccessed = pd.DataFrame(dict_data_validate)

In [ ]:
df_validate_preproccessed.to_csv("preproccessed_dataset/preprocessed_validate.csv")